## COMP5623 Coursework on Image Classification with Convolutional Neural Networks 

Starter code.

In [0]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from  torch.utils.data import Dataset

from sklearn.metrics import confusion_matrix
from skimage import io, transform

import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import csv
import os
import math
import cv2

### Part I

The first part of the assignment is to build a CNN and train it on a subset of the ImageNet dataset. We will first create a dataframe with all the references to the images and their labels.

To download the images into your work environment, clone into a git respository containing the images.

In [124]:
! git clone https://github.com/MohammedAlghamdi/imagenet10.git

fatal: destination path 'imagenet10' already exists and is not an empty directory.


Check that the repository is there:

In [125]:
! ls

imagenet10  my_mnist_model.pt  sample_data


In [0]:
root_dir = "imagenet10/train_set/"
class_names = [
  "baboon",
  "banana",
  "canoe",
  "cat",
  "desk",
  "drill",
  "dumbbell",
  "football",
  "mug",
  "orange",
]

A helper function for reading in images and assigning labels.

In [0]:
def get_meta(root_dir, dirs):
    """ Fetches the meta data for all the images and assigns labels.
    """
    paths, classes = [], []
    for i, dir_ in enumerate(dirs):
        for entry in os.scandir(root_dir + dir_):
            if (entry.is_file()):
                paths.append(entry.path)
                classes.append(i)
                
    return paths, classes

Now we create a dataframe using all the data.

In [0]:
# Benign images we will assign class 0, and malignant as 1
paths, classes = get_meta(root_dir, class_names)

data = {
    'path': paths,
    'class': classes
}

data_df = pd.DataFrame(data, columns=['path', 'class'])
data_df = data_df.sample(frac=1).reset_index(drop=True) # Shuffles the data

View some sample data.

In [129]:
print("Found", len(data_df), "images.")
data_df.head()

Found 9000 images.


,path,class
0,imagenet10/train_set/drill/n03239726_15082.JPEG,5
1,imagenet10/train_set/drill/n03239726_1624.JPEG,5
2,imagenet10/train_set/drill/n03239726_8850.JPEG,5
3,imagenet10/train_set/banana/n07753592_10986.JPEG,1
4,imagenet10/train_set/canoe/n02951358_3826.JPEG,2


Now we will create the Dataset class.

In [0]:
class ImageNet10(Dataset):
    """ ImageNet10 dataset. """

    def __init__(self, df, transform=None):
        """
        Args:
            image_dir (string): Directory with all the images
            df (DataFrame object): Dataframe containing the images, paths and classes
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load image from path and get label
        x = Image.open(self.df['path'][index])
        try:
          x = x.convert('RGB') # To deal with some grayscale images in the data
        except:
          pass
        y = torch.tensor(int(self.df['class'][index]))

        if self.transform:
            x = self.transform(x)

        return x, y

Compute what we should normalise the dataset to.

In [0]:
def compute_img_mean_std(image_paths):
    """
        Author: @xinruizhuang. Computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 224, 224
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means, stdevs


In [132]:
norm_mean, norm_std = compute_img_mean_std(paths)

100%|██████████| 9000/9000 [00:46<00:00, 205.57it/s]


(224, 224, 3, 9000)
normMean = [0.5228344, 0.47988218, 0.40605018]
normStd = [0.29770824, 0.28884, 0.31178245]


Now let's create the transforms to normalise and turn our data into tensors.

In [0]:
data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std),
    ])

# data augmentation

In [0]:
data_transform1 = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomCrop(96),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std),
    ])

Let's split the data into train and test sets and instantiate our new ISIC_Dataset objects.

In [0]:
#百分之70给了train，10给valid，20给test
train_split = 0.70 # Defines the ratio of train/valid/test data.
valid_split = 0.10
#demo_split = 2 / 9000

train_size = int(len(data_df)*train_split)
valid_size = int(len(data_df)*valid_split)
#demo_size = int(len(data_df)*demo_split)

ins_dataset_train = ImageNet10(
    df=data_df[:train_size],
    transform=data_transform,
)

ins_dataset_valid = ImageNet10(
    df=data_df[train_size:(train_size + valid_size)].reset_index(drop=True),
    transform=data_transform,
)

ins_dataset_test = ImageNet10(
    df=data_df[(train_size + valid_size):].reset_index(drop=True),
    transform=data_transform,
)
# ins_dataset_demo = ImageNet10(
#     df=data_df[:demo_size],
#     transform=data_transform,
# )

In [0]:
# #ins_dataset_train[0]
# img=ins_dataset_test[0][0][1].numpy()
# #img.shape
# #img
# plt.imshow(img)
# # print( img.shape)
# #img=img.numpy()
# # #img=img.reshape(28,28)
# # plt.imshow(img,cmap='gray')
# # plt.show()
# # #ins_dataset_train[0][0]

# **Data Loader**

In [0]:
train_loader = torch.utils.data.DataLoader(
    ins_dataset_train,
    batch_size=16,
    shuffle=True,
    num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    ins_dataset_test,
    batch_size=24,
    shuffle=True,
    num_workers=2
)

valid_loader = torch.utils.data.DataLoader(
    ins_dataset_valid,
    batch_size=24,
    shuffle=True,
    num_workers=2
)

# demo_loader = torch.utils.data.DataLoader(
#     ins_dataset_demo,
#     batch_size=1,
#     shuffle=True,
#     num_workers=2
# )




# > Calcuate the final convolution output



In [0]:

def calOutput():
  
  first_layer = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3)
          )
  second_layer = nn.Sequential(
            nn.Conv2d(16, 24, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3)
          )
  third_layer = nn.Sequential(
            nn.Conv2d(24, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3)
          )
  forth_layer = nn.Sequential(
          nn.Conv2d(32, 40, 5),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )

  output = first_layer(input)
  output = second_layer(output)
  output = third_layer(output)
  output = forth_layer(output)
  print(output.shape)

# Model1 for 2 layers

In [0]:
# Convolutional neural network
class ConvNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.first_layer = nn.Sequential(
          nn.Conv2d(3, 16, 3, 1, 1),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.second_layer = nn.Sequential(
          nn.Conv2d(16, 24, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        # self.third_layer = nn.Sequential(
        #   nn.Conv2d(24, 32, 4),
        #   nn.ReLU(),
        #   nn.MaxPool2d(2),
        #   nn.Dropout2d(0.3)
        # )

  
        # Add network layers here

        self.fc1 = nn.Linear(24 * 62 * 62, 512)
        self.fc2 = nn.Linear(512, 10)
        #self.final = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = self.first_layer(x)
        out1 = self.second_layer(out)
        #out2 = self.third_layer(out1)
        out3 = out1.reshape(out1.size(0), -1)
        out4 = self.fc1(out3)
        out5 = self.fc2(out4)        
        #out6 = self.final(out5)
        return out5

# Model for 3 Layers

In [0]:
# Convolutional neural network
class ConvNet3(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet3, self).__init__()
        self.first_layer = nn.Sequential(
          nn.Conv2d(3, 16, 3, 1, 1),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.second_layer = nn.Sequential(
          nn.Conv2d(16, 24, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.third_layer = nn.Sequential(
          nn.Conv2d(24, 32, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )

  
        # Add network layers here

        self.fc1 = nn.Linear(32 * 29 * 29, 512)
        self.fc2 = nn.Linear(512, 10)
        #self.final = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = self.first_layer(x)
        out = self.second_layer(out)
        out = self.third_layer(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)        
        #out6 = self.final(out5)
        return out

# Model for 4 layers

In [0]:
# Convolutional neural network
class ConvNet4(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet4, self).__init__()
        self.first_layer = nn.Sequential(
          nn.Conv2d(3, 16, 3),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.5)
        )
        
        self.second_layer = nn.Sequential(
          nn.Conv2d(16, 30, 5),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.5)
        )
        self.third_layer = nn.Sequential(
          nn.Conv2d(30, 48, 5),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.5)
        )
        self.forth_layer = nn.Sequential(
          nn.Conv2d(48, 60, 5),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.5)
        )

  
        # Add network layers here

        self.fc1 = nn.Linear(8640, 4096)
        self.fc5 = nn.Linear(4096, 512)
        # self.fc2 = nn.Linear(4096, 2000)
        # self.fc3 = nn.Linear(2000, 512)
        self.fc4 = nn.Linear(512, 10)
        #self.final = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = self.first_layer(x)
        #self.feature1 = out
        out = self.second_layer(out)
        out = self.third_layer(out)
        out = self.forth_layer(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc5(out)
        # out = self.fc2(out)
        # out = self.fc3(out)
        out = self.fc4(out)        
        #out6 = self.final(out5)
        return out

# Model for 5 layers

In [0]:
# Convolutional neural network
class ConvNet5(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet5, self).__init__()
        self.first_layer = nn.Sequential(
          nn.Conv2d(3, 16, 3),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.second_layer = nn.Sequential(
          nn.Conv2d(16, 24, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.third_layer = nn.Sequential(
          nn.Conv2d(24, 32, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.forth_layer = nn.Sequential(
          nn.Conv2d(32, 40, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.fifth_layer = nn.Sequential(
          nn.Conv2d(40, 48, 4),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )

  
        # Add network layers here

        self.fc1 = nn.Linear(48 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 10)
        #self.final = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = self.first_layer(x)
        out = self.second_layer(out)
        out = self.third_layer(out)
        out = self.forth_layer(out)
        out = self.fifth_layer(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)        
        #out6 = self.final(out5)
        return out

# Model for change kernel size for 4 layers

In [0]:
# Convolutional neural network
class ConvNet_resize(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet_resize, self).__init__()
        self.first_layer = nn.Sequential(
          nn.Conv2d(3, 16, 6),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.second_layer = nn.Sequential(
          nn.Conv2d(16, 24, 6),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.third_layer = nn.Sequential(
          nn.Conv2d(24, 32, 6),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )
        self.forth_layer = nn.Sequential(
          nn.Conv2d(32, 40, 6),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout2d(0.3)
        )

  
        # Add network layers here

        self.fc1 = nn.Linear(5760, 512)
        self.fc2 = nn.Linear(512, 10)
        #self.final = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = self.first_layer(x)
        self.feature1 = out
        out = self.second_layer(out)
        out = self.third_layer(out)
        out = self.forth_layer(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)        
        #out6 = self.final(out5)
        return out

# initialization

In [0]:
torch.cuda.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_cpu = torch.device('cpu')
#device
model_gpu = ConvNet4().to(device)
#from torch import optim
criterion = nn.CrossEntropyLoss()

# Stochastic gradient descent
optimizer = optim.SGD(model_gpu.parameters(), lr=0.001, momentum=0.9)

# Fliter Visual

In [0]:
def filterVisual():
  figure = plt.figure(figsize = (10, 10))
  k = 0

  for i in range(16):
    
    filter_mix = np.zeros(shape = [3,3])
    for j in range(3):
      k = k + 1
      figure.add_subplot(8, 6, k)
      filter_mix += model_gpu.first_layer[0].weight.data.cpu().numpy()[ i, j, :, :]
      plt.imshow(filter_mix, cmap = "gray")
  plt.show()

In [0]:
#filterVisual()

In [0]:

#model = ConvNet()
#model
#model.conv1
#model.conv1.weight

In [0]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device_cpu = torch.device('cpu')
# #device
# model_gpu = ConvNet4().to(device)
# #from torch import optim
# criterion = nn.CrossEntropyLoss()

# # Stochastic gradient descent
# optimizer = optim.SGD(model_gpu.parameters(), lr=0.001, momentum=0.9)

# Training

In [0]:
array1 = []

In [0]:

array2 = []
# array2 = [2]
# array1.extend(array2)
# array1.append(1)
# array1

In [0]:
#import timeit
# Device configuration - defaults to CPU unless GPU is available on device

def train_model_epochs2(num_epochs):
    """ Copy of function train_model_epochs num_epochsbut explicitly copying data to device 
        during training. 
    """
#    for epoch in range(num_epochs):
    running_loss = 0.0
    sum_running_loss = 0
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        
        # Explicitly specifies that data is to be copied onto the device!
        images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
        labels = labels.to(device)  # <----------- variables still exist on CPU

        optimizer.zero_grad()
        outputs = model_gpu(images)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('train_Epoch / Batch [%d / %d] - Loss: %.3f' %
                  (num_epochs + 1, i + 1, running_loss / 100))
            sum_running_loss = sum_running_loss + (running_loss / 100)
            running_loss = 0.0
    array1.append(sum_running_loss / 3)
      
                

In [0]:
def valid_model_epochs3(num_epochs):
    """ Copy of function train_model_epochs but explicitly copying data to device 
        during training. 
    """
    running_loss = 0.0
    sum_running_loss = 0
    for i, data in enumerate(valid_loader, 0):
        images, labels = data

        # Explicitly specifies that data is to be copied onto the device!
        images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
        labels = labels.to(device)  # <----------- variables still exist on CPU

        #optimizer.zero_grad()
        outputs = model_gpu(images)

        loss = criterion(outputs, labels)
        #loss.backward()

        #optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:    # print every 100 mini-batches
            print('valid_Epoch / Batch [%d / %d] - Loss: %.3f' %
                  (num_epochs + 1, i + 1, running_loss / 10))
            sum_running_loss = sum_running_loss + (running_loss / 10)
            
            running_loss = 0.0
    array2.append(sum_running_loss / 3)

In [0]:
def demo():
  for num_epochs in range(0,30):
    if(num_epochs == 12):
      #filterVisual()
      #optimizer = optim.SGD(model_gpu.parameters(), lr=0.0005, momentum=0.9)
      print("运行了")
    train_model_epochs2(num_epochs)
    #print("保存")
    #torch.save(model_gpu, './my_mnist_model.pt') 
    print("拿去验证")
    valid_model_epochs3(num_epochs)
    print("下个回合")

In [0]:
demo()

In [0]:
array1

In [0]:
array2

In [0]:
def drawplot(): 
  x = np.arange(0,30)
  y = array1
  y1 = array2
  
  plt.figure()
  plt.plot(x, y,  "b--", linewidth=1)
  plt.plot(x, y1, "r--", linewidth = 1)
  plt.xlabel("epoch")
  plt.ylabel("loss")
  plt.title("loss for finding converge")


In [0]:
drawplot()

In [0]:

correct = 0
total = 0
label1 = torch.tensor([], dtype = torch.long)
predicated1 = torch.tensor([], dtype = torch.long)
#predicated1 = torch.tensor(0)
# Why don't we need gradients? What happens if we do include gradients?
with torch.no_grad():
    
    # Iterate over the test set
    for data in test_loader:
        images, labels = data
        images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
        labels = labels.to(device)  # <----------- variables still exist on CPU

        outputs = model_gpu(images)

        
        # torch.max is an argmax operation
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        label = labels.to(device_cpu)
        predicated = predicted.to(device_cpu)
        
        
        label1 = torch.cat([label1, label], dim = 0)
        predicated1 = torch.cat([predicated1, predicated], dim = 0)

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

In [0]:
images.cpu().detach().clone().numpy
conv1_activs = model_gpu.first_layer[0].forward(images)
conv1_activs_relu = model_gpu.first_layer[1].forward(conv1_activs)
conv1_activs_pooling = model_gpu.first_layer[2].forward(conv1_activs_relu)
conv1_activs_drop = model_gpu.first_layer[3].forward(conv1_activs_pooling)

conv2_activs = model_gpu.second_layer[0].forward(conv1_activs_drop)
conv2_activs_relu = model_gpu.second_layer[1].forward(conv2_activs)
conv2_activs_pooling = model_gpu.second_layer[2].forward(conv2_activs_relu)
conv2_activs_drop = model_gpu.second_layer[3].forward(conv2_activs_pooling)

conv3_activs = model_gpu.third_layer[0].forward(conv2_activs_drop)
conv3_activs_relu = model_gpu.third_layer[1].forward(conv3_activs)
conv3_activs_pooling = model_gpu.third_layer[2].forward(conv3_activs_relu)
conv3_activs_drop = model_gpu.third_layer[3].forward(conv3_activs_pooling)

conv4_activs = model_gpu.forth_layer[0].forward(conv3_activs_drop)
# conv4_activs_relu = model_gpu.forth_layer[1].forward(conv4_activs)
# conv4_activs_pooling = model_gpu.forth_layer[2].forward(conv3_activs_relu)
# conv4_activs_drop = model_gpu.forth_layer[3].forward(conv3_activs_pooling)
#print(conv1_activs)
figure = plt.figure(figsize = (30, 40))
k = 0
for j in range(0, 2):
  
  for i in range(0, 4):
    k += 1
    figure.add_subplot(8,4,k)
    img=conv1_activs[j][i].cpu().detach().clone().numpy()
    plt.imshow(img, cmap = "gray")
    
  for i in range(0, 4):
    k += 1
    figure.add_subplot(8, 4, k)
    img1=conv2_activs[j][i].cpu().detach().clone().numpy()
    plt.imshow(img1, cmap = "gray")
  for i in range(0, 4):
    k += 1
    figure.add_subplot(8, 4, k)
    img2=conv3_activs[j][i].cpu().detach().clone().numpy()
    plt.imshow(img2, cmap = "gray")
  for i in range(0, 4):
    k += 1
    figure.add_subplot(8, 4, k)
    img3=conv4_activs[j][i].cpu().detach().clone().numpy()
    plt.imshow(img3, cmap = "gray")
#print(img.shape)
# labels
# images
# images.cpu().detach().clone().numpy()


In [0]:
#label1

In [0]:
#labels

In [286]:
torch.save(model_gpu, './my_mnist_model.pt') 

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ConvNet4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [287]:

#labels是正确的分类，predicted是预测的，放到一起，都是test里面的
cm = confusion_matrix(label1, predicated1)
cm

array([[115,   5,  15,  29,   2,   4,   3,   9,   4,   0],
       [ 13,  48,  14,  20,   9,   6,   6,   7,  11,  37],
       [ 18,   2,  94,  15,  21,   6,  11,   8,   4,   1],
       [ 58,   3,  11,  75,   2,   5,  11,   9,   2,   0],
       [  7,   8,  28,   4,  75,  21,  11,   9,  16,   1],
       [ 12,   9,  11,   4,  25,  77,  19,  12,   6,   7],
       [ 20,   7,  16,  13,  34,  14,  49,  13,  13,   2],
       [ 31,   9,  34,  24,  14,   9,  17,  33,   9,   1],
       [ 23,   7,   9,   9,  31,  18,  20,  12,  52,   2],
       [  9,  62,   6,   5,   5,   6,   3,   1,   6,  77]])

In [0]:
classes = np.arange(0,10)

In [0]:
import itertools

def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix very prettily.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)

    # Specify the tick marks and axis text
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    # The data formatting
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Print the text of the matrix, adjusting text colour for display
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [0]:
plot_confusion_matrix(cm, classes)

In [0]:
filterVisual()
# figure = plt.figure(figsize = (16, 16))
# k = 0
# for i in range(16):
#   k = k + 1
#   filter_mix = np.zeros(shape = [3,3])
#   for j in range(3):
#     figure.add_subplot(4, 4, k)
#     filter_mix += model_gpu.first_layer[0].weight.data.cpu().numpy()[i, j, :, :]
#   plt.imshow(filter_mix, cmap = "hot")
# plt.show()
